<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/revisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install as libraries necessárias

In [ ]:
!pip install pycaret

In [ ]:
!pip install shap

### Carregar as libraries

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt # Gráficos

import seaborn as sns
sns.set_theme(style = 'ticks', palette = 'pastel')

from pycaret import regression, classification # Canivete suiço para Machine Learning
import shap

from collections import Counter

import tensorflow as tf
tf.__version__

In [ ]:
from pycaret.utils import enable_colab
enable_colab()

### Configurações de ambiente

In [ ]:
pd_options = pd.options.display # Objeto para configuração do ambiente: número de linhas, colunas, precisão e etc

In [ ]:
pd_options.max_columns = 50 # número máximo de colunas
pd_options.max_rows = 50 # número máximo de linhas
pd_options.precision = 3 # número de casas decimais

### Facets - DataViz

In [ ]:
from IPython.core.display import display, HTML
import base64

In [ ]:
!pip install facets-overview==1.0.0

In [ ]:
from facets_overview.feature_statistics_generator import FeatureStatisticsGenerator

### Carregar os dados de treinamento e teste

In [ ]:
url_T = ''
url_V = ''